In [1]:
import os
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import matplotlib as mpl
%matplotlib inline
#sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

pd.set_option('display.max_columns', 500)
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_curve, confusion_matrix,average_precision_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold, GridSearchCV
from sklearn.feature_selection import RFECV, RFE
from sklearn.metrics import roc_curve, auc, recall_score, f1_score, precision_score,roc_auc_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
rf = RandomForestClassifier()
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

In [4]:
one_above = pd.read_csv('/Users/yichingchan/Documents/91APP/one_above.csv')
twice_above = pd.read_csv('/Users/yichingchan/Documents/91APP/twice_above.csv')
three_above = pd.read_csv('/Users/yichingchan/Documents/91APP/three_above.csv')
four_above = pd.read_csv('/Users/yichingchan/Documents/91APP/four_above.csv')
five_above = pd.read_csv('/Users/yichingchan/Documents/91APP/five_above.csv')


In [55]:
def binary_classifier(df, model):
    
    #split dataset
    train_Y = df['retain']
    print(train_Y.value_counts())
    print()

    drop_list =['MemberId', 'count', 'label', 'ctrbn', 'retain']
    train_X = df.drop(drop_list ,axis=1)
    column_name = train_X.columns.tolist()

    train_X = train_X.values
    train_X, test_X, train_Y, test_Y = train_test_split(train_X, train_Y, test_size=0.2)
    
    #model fit
    model = model
    model.fit(train_X, train_Y)

    #cross validation score
    #score = cross_val_score(rf, train_X, train_Y, cv=5).mean()
    #print("cross validation score: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))

    ##predict
    pred_Y = model.predict(test_X)
    probs_Y = model.predict_proba(test_X)
    probs_Y = probs_Y[:, 1] ## keep probabilities for the positive outcome only    
    
    #evaluation
    accy = accuracy_score(test_Y, pred_Y)
    f1 = f1_score(test_Y, pred_Y)
    recall = recall_score(test_Y, pred_Y, average='macro')
    precision =precision_score(test_Y, pred_Y)
    auc = roc_auc_score(test_Y, probs_Y)
    print('evaluation without RFECV: \n accy: %0.3f, \n auc: %0.3f, \n f1: %0.3f, \n recall: %0.3f, \n precision: %0.3f' % (accy, auc, f1, recall, precision))
    print()
    
    #feature importance
    try:
        feature_importances = pd.Series(model.feature_importances_, index=column_name)
    except:
        feature_importances = None
    
    #confusion matrix
    labels = np.unique(test_Y)
    conf_m = confusion_matrix(test_Y, pred_Y, labels=labels)
    conf_m = pd.DataFrame(conf_m, index=labels, columns=labels)
    print('confusion matrix :', conf_m)
    print()
    
    # calculate precision-recall curve
    #precision, recall, thresholds = precision_recall_curve(test_Y, probs_Y)
    #f1 = f1_score(test_Y, pred_Y) # calculate F1 score
    #PRauc = auc(recall, precision) # calculate precision-recall AUC
    ap = average_precision_score(test_Y, probs_Y) # calculate average precision score
    print('ap: %0.3f' % ap )
    
    
    
    return pred_Y, probs_Y, accy, f1, recall, precision, conf_m, ap, feature_importances
    

In [74]:
model = LogisticRegression()
d = dict()

accy_lst = []
f1_lst = []
recall_lst = []
precision_lst = []
ap_lst = []
#conf_m_lst = []


df_lst = [one_above, twice_above, three_above, four_above, five_above]
for df in df_lst:
    pred_Y, probs_Y, accy, f1, recall, precision, conf_m, ap, feature_importances = binary_classifier(df, model)
    accy_lst.append(accy)
    f1_lst.append(f1)
    recall_lst.append(recall)
    precision_lst.append(precision)
    ap_lst.append(ap)
    #conf_m_lst.append(conf_m)
    
    
d['LR_accy'] = accy_lst
d['LR_f1'] = f1_lst
d['LR_recall'] = recall_lst
d['LR_precision'] = precision_lst
d['LR_ap'] = ap_lst
#LR['conf_m'] = conf_m_lst
LR_matrix = pd.DataFrame(d)
LR_matrix.index = ['oneMore', 'twoMore', 'threeMore', 'fourMore', 'fiveMore']


False    184348
True      72142
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.757, 
 auc: 0.729, 
 f1: 0.392, 
 recall: 0.612, 
 precision: 0.670

confusion matrix :        False  True 
False  34826   1980
True   10471   4021

ap: 0.555
False    47381
True     37720
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.657, 
 auc: 0.700, 
 f1: 0.571, 
 recall: 0.642, 
 precision: 0.637

confusion matrix :        False  True 
False   7292   2211
True    3631   3887

ap: 0.658
True     21272
False    16860
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.642, 
 auc: 0.690, 
 f1: 0.698, 
 recall: 0.630, 
 precision: 0.656

confusion matrix :        False  True 
False   1756   1652
True    1075   3144

ap: 0.725
True     12650
False     7156
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.679, 
 auc: 0.678, 
 f1: 0.777, 
 recall: 0.604, 
 precision: 0.705

confusion matrix :        False  True 
False    485    924
True     346   22

In [83]:
model = RandomForestClassifier()
d = dict()

accy_lst = []
f1_lst = []
recall_lst = []
precision_lst = []
ap_lst = []
#conf_m_lst = []
feature_lst = []

df_lst = [one_above, twice_above, three_above, four_above, five_above]
for df in df_lst:
    pred_Y, probs_Y, accy, f1, recall, precision, conf_m, ap, feature_importances = binary_classifier(df, model)
    accy_lst.append(accy)
    f1_lst.append(f1)
    recall_lst.append(recall)
    precision_lst.append(precision)
    ap_lst.append(ap)
    #conf_m_lst.append(conf_m)
    feature_lst.append(feature_importances.to_dict())
    
d['RF_accy'] = accy_lst
d['RF_f1'] = f1_lst
d['RF_recall'] = recall_lst
d['RF_precision'] = precision_lst
d['RF_ap'] = ap_lst
#LR['conf_m'] = conf_m_lst
RF_matrix = pd.DataFrame(d)
RF_matrix.index = ['oneMore', 'twoMore', 'threeMore', 'fourMore', 'fiveMore']

RFfeature_rank = pd.DataFrame(feature_lst)
RFfeature_mean_score = RFfeature_rank.mean().sort_values(ascending=False)

False    184348
True      72142
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.725, 
 auc: 0.667, 
 f1: 0.378, 
 recall: 0.595, 
 precision: 0.514

confusion matrix :        False  True 
False  32937   4044
True   10044   4273

ap: 0.438
False    47381
True     37720
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.620, 
 auc: 0.642, 
 f1: 0.506, 
 recall: 0.602, 
 precision: 0.598

confusion matrix :        False  True 
False   7251   2222
True    4242   3306

ap: 0.575
True     21272
False    16860
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.606, 
 auc: 0.642, 
 f1: 0.635, 
 recall: 0.605, 
 precision: 0.657

confusion matrix :        False  True 
False   2010   1365
True    1638   2614

ap: 0.670
True     12650
False     7156
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.622, 
 auc: 0.623, 
 f1: 0.711, 
 recall: 0.581, 
 precision: 0.687

confusion matrix :        False  True 
False    623    837
True     661   18

In [84]:
model = XGBClassifier()
d = dict()

accy_lst = []
f1_lst = []
recall_lst = []
precision_lst = []
ap_lst = []
#conf_m_lst = []
feature_lst = []

df_lst = [one_above, twice_above, three_above, four_above, five_above]
for df in df_lst:
    pred_Y, probs_Y, accy, f1, recall, precision, conf_m, ap, feature_importances = binary_classifier(df, model)
    accy_lst.append(accy)
    f1_lst.append(f1)
    recall_lst.append(recall)
    precision_lst.append(precision)
    ap_lst.append(ap)
    #conf_m_lst.append(conf_m)
    feature_lst.append(feature_importances.to_dict())
    
d['XGB_accy'] = accy_lst
d['XGB_f1'] = f1_lst
d['XGB_recall'] = recall_lst
d['XGB_precision'] = precision_lst
d['XGB_ap'] = ap_lst
#LR['conf_m'] = conf_m_lst
XGB_matrix = pd.DataFrame(d)
XGB_matrix.index = ['oneMore', 'twoMore', 'threeMore', 'fourMore', 'fiveMore']

XGBfeature_rank = pd.DataFrame(feature_lst)
XGBfeature_mean_score = XGBfeature_rank.mean().sort_values(ascending=False)

False    184348
True      72142
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.762, 
 auc: 0.729, 
 f1: 0.419, 
 recall: 0.622, 
 precision: 0.658

confusion matrix :        False  True 
False  34679   2287
True    9934   4398

ap: 0.554
False    47381
True     37720
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.664, 
 auc: 0.710, 
 f1: 0.574, 
 recall: 0.648, 
 precision: 0.651

confusion matrix :        False  True 
False   7470   2055
True    3656   3840

ap: 0.669
True     21272
False    16860
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.654, 
 auc: 0.704, 
 f1: 0.707, 
 recall: 0.641, 
 precision: 0.669

confusion matrix :        False  True 
False   1797   1576
True    1065   3189

ap: 0.751
True     12650
False     7156
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.675, 
 auc: 0.708, 
 f1: 0.774, 
 recall: 0.600, 
 precision: 0.692

confusion matrix :        False  True 
False    467    983
True     305   22

In [85]:
model = GradientBoostingClassifier()
d = dict()

accy_lst = []
f1_lst = []
recall_lst = []
precision_lst = []
ap_lst = []
#conf_m_lst = []
feature_lst = []

df_lst = [one_above, twice_above, three_above, four_above, five_above]
for df in df_lst:
    pred_Y, probs_Y, accy, f1, recall, precision, conf_m, ap, feature_importances = binary_classifier(df, model)
    accy_lst.append(accy)
    f1_lst.append(f1)
    recall_lst.append(recall)
    precision_lst.append(precision)
    ap_lst.append(ap)
    #conf_m_lst.append(conf_m)
    feature_lst.append(feature_importances.to_dict())
    
d['GB_accy'] = accy_lst
d['GB_f1'] = f1_lst
d['GB_recall'] = recall_lst
d['GB_precision'] = precision_lst
d['GB_ap'] = ap_lst
#LR['conf_m'] = conf_m_lst
GB_matrix = pd.DataFrame(d)
GB_matrix.index = ['oneMore', 'twoMore', 'threeMore', 'fourMore', 'fiveMore']

GBfeature_rank = pd.DataFrame(feature_lst)
GBfeature_mean_score = GBfeature_rank.mean().sort_values(ascending=False)

False    184348
True      72142
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.758, 
 auc: 0.731, 
 f1: 0.415, 
 recall: 0.620, 
 precision: 0.653

confusion matrix :        False  True 
False  34491   2340
True   10070   4397

ap: 0.558
False    47381
True     37720
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.658, 
 auc: 0.704, 
 f1: 0.564, 
 recall: 0.641, 
 precision: 0.640

confusion matrix :        False  True 
False   7437   2119
True    3704   3761

ap: 0.664
True     21272
False    16860
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.640, 
 auc: 0.690, 
 f1: 0.693, 
 recall: 0.629, 
 precision: 0.658

confusion matrix :        False  True 
False   1787   1610
True    1132   3098

ap: 0.731
True     12650
False     7156
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.687, 
 auc: 0.711, 
 f1: 0.781, 
 recall: 0.614, 
 precision: 0.707

confusion matrix :        False  True 
False    506    919
True     321   22

In [86]:
model = GaussianNB()
d = dict()

accy_lst = []
f1_lst = []
recall_lst = []
precision_lst = []
ap_lst = []
#conf_m_lst = []

df_lst = [one_above, twice_above, three_above, four_above, five_above]
for df in df_lst:
    pred_Y, probs_Y, accy, f1, recall, precision, conf_m, ap, feature_importances = binary_classifier(df, model)
    accy_lst.append(accy)
    f1_lst.append(f1)
    recall_lst.append(recall)
    precision_lst.append(precision)
    ap_lst.append(ap)
    #conf_m_lst.append(conf_m)
    
d['NB_accy'] = accy_lst
d['NB_f1'] = f1_lst
d['NB_recall'] = recall_lst
d['NB_precision'] = precision_lst
d['NB_ap'] = ap_lst
#LR['conf_m'] = conf_m_lst
NB_matrix = pd.DataFrame(d)
NB_matrix.index = ['oneMore', 'twoMore', 'threeMore', 'fourMore', 'fiveMore']

False    184348
True      72142
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.726, 
 auc: 0.685, 
 f1: 0.379, 
 recall: 0.596, 
 precision: 0.533

confusion matrix :        False  True 
False  32971   3759
True   10279   4289

ap: 0.470
False    47381
True     37720
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.618, 
 auc: 0.668, 
 f1: 0.415, 
 recall: 0.585, 
 precision: 0.637

confusion matrix :        False  True 
False   8212   1316
True    5187   2306

ap: 0.602
True     21272
False    16860
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.567, 
 auc: 0.660, 
 f1: 0.497, 
 recall: 0.592, 
 precision: 0.710

confusion matrix :        False  True 
False   2693    666
True    2638   1630

ap: 0.697
True     12650
False     7156
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.626, 
 auc: 0.672, 
 f1: 0.683, 
 recall: 0.625, 
 precision: 0.746

confusion matrix :        False  True 
False    886    544
True     938   15

In [87]:
df_matrix = pd.concat([LR_matrix, RF_matrix, XGB_matrix, GB_matrix, NB_matrix ], axis=1)
df_matrix


,LR_accy,LR_f1,LR_recall,LR_precision,LR_ap,RF_accy,RF_f1,RF_recall,RF_precision,RF_ap,XGB_accy,XGB_f1,XGB_recall,XGB_precision,XGB_ap,GB_accy,GB_f1,GB_recall,GB_precision,GB_ap,NB_accy,NB_f1,NB_recall,NB_precision,NB_ap
oneMore,0.757281,0.392427,0.611834,0.670055,0.554784,0.725369,0.377574,0.594551,0.513767,0.437724,0.761765,0.418518,0.622499,0.657891,0.553845,0.758080,0.414733,0.620200,0.652664,0.557535,0.726344,0.379289,0.596036,0.532927,0.470474
twoMore,0.656777,0.570946,0.642181,0.637422,0.658319,0.620234,0.505659,0.601718,0.598046,0.574652,0.664473,0.573520,0.648263,0.651399,0.668663,0.657893,0.563657,0.641036,0.639626,0.663763,0.617943,0.414935,0.584817,0.636665,0.601654
threeMore,0.642454,0.697504,0.630229,0.655546,0.724890,0.606267,0.635160,0.605163,0.656949,0.669915,0.653730,0.707174,0.641204,0.669255,0.750689,0.640488,0.693220,0.629220,0.658029,0.730587,0.566802,0.496648,0.591819,0.709930,0.696861
fourMore,0.679455,0.776566,0.604344,0.704887,0.777099,0.621908,0.710811,0.581262,0.687453,0.716585,0.674912,0.774114,0.600326,0.691850,0.803106,0.687027,0.781382,0.614280,0.706858,0.808323,0.625946,0.682655,0.624561,0.745557,0.762416
fiveMore,0.717131,0.822352,0.571724,0.733631,0.809867,0.673307,0.779042,0.573583,0.742156,0.783785,0.711819,0.817391,0.574145,0.728864,0.838377,0.723772,0.825308,0.582780,0.747465,0.827143,0.678176,0.766613,0.626367,0.776333,0.816737


In [89]:
mod = ['LR_', 'RF_', 'XGB_', 'GB_', 'NB_']
eva = ['accy', 'f1', 'recall', 'precision', 'ap']
accy = pd.DataFrame()
f1 = pd.DataFrame()
recall = pd.DataFrame()
precision = pd.DataFrame()
ap = pd.DataFrame()

df = [accy, f1, recall, precision, ap]

for name in mod:
    for index in range(len(eva)):
        sr = name + eva[index]
        df[index][sr] = df_matrix[sr]
    

In [90]:
accy

,LR_accy,RF_accy,XGB_accy,GB_accy,NB_accy
oneMore,0.757281,0.725369,0.761765,0.758080,0.726344
twoMore,0.656777,0.620234,0.664473,0.657893,0.617943
threeMore,0.642454,0.606267,0.653730,0.640488,0.566802
fourMore,0.679455,0.621908,0.674912,0.687027,0.625946
fiveMore,0.717131,0.673307,0.711819,0.723772,0.678176


In [91]:
f1

,LR_f1,RF_f1,XGB_f1,GB_f1,NB_f1
oneMore,0.392427,0.377574,0.418518,0.414733,0.379289
twoMore,0.570946,0.505659,0.573520,0.563657,0.414935
threeMore,0.697504,0.635160,0.707174,0.693220,0.496648
fourMore,0.776566,0.710811,0.774114,0.781382,0.682655
fiveMore,0.822352,0.779042,0.817391,0.825308,0.766613


In [92]:
recall

,LR_recall,RF_recall,XGB_recall,GB_recall,NB_recall
oneMore,0.611834,0.594551,0.622499,0.620200,0.596036
twoMore,0.642181,0.601718,0.648263,0.641036,0.584817
threeMore,0.630229,0.605163,0.641204,0.629220,0.591819
fourMore,0.604344,0.581262,0.600326,0.614280,0.624561
fiveMore,0.571724,0.573583,0.574145,0.582780,0.626367


In [93]:
precision

,LR_precision,RF_precision,XGB_precision,GB_precision,NB_precision
oneMore,0.670055,0.513767,0.657891,0.652664,0.532927
twoMore,0.637422,0.598046,0.651399,0.639626,0.636665
threeMore,0.655546,0.656949,0.669255,0.658029,0.709930
fourMore,0.704887,0.687453,0.691850,0.706858,0.745557
fiveMore,0.733631,0.742156,0.728864,0.747465,0.776333


In [94]:
ap

,LR_ap,RF_ap,XGB_ap,GB_ap,NB_ap
oneMore,0.554784,0.437724,0.553845,0.557535,0.470474
twoMore,0.658319,0.574652,0.668663,0.663763,0.601654
threeMore,0.724890,0.669915,0.750689,0.730587,0.696861
fourMore,0.777099,0.716585,0.803106,0.808323,0.762416
fiveMore,0.809867,0.783785,0.838377,0.827143,0.816737


In [102]:
feaRank = []
feaRank.append(RFfeature_mean_score.to_dict())
feaRank.append(XGBfeature_mean_score.to_dict())
feaRank.append(GBfeature_mean_score.to_dict())
featureRanking = pd.DataFrame(feaRank)
featureRanking.index = ['RF', 'XGB', 'GB']
featureRanking.transpose()

,RF,XGB,GB
DeliverTypeDef_Family,0.008300,0.000000,0.000196
DeliverTypeDef_Home,0.004952,0.000000,0.000062
DeliverTypeDef_SevenEleven,0.008695,0.000000,0.000195
ECouponDiscount,0.015212,0.020247,0.011103
PayProfileTypeDef_ATM,0.000016,0.000000,0.000038
PayProfileTypeDef_CreditCardOnce,0.004769,0.009585,0.000567
PayProfileTypeDef_Family,0.008612,0.004643,0.000224
PayProfileTypeDef_LinePay,0.000000,0.000000,0.000000
PayProfileTypeDef_SevenEleven,0.008307,0.001945,0.000362
PromotionDiscount,0.019981,0.010949,0.003774


### Dummy Columns Redfine: slightly better than the previous results 

In [121]:
one_above_sum = pd.read_csv('/Users/yichingchan/Documents/91APP/one_above_sum.csv')
twice_above_sum = pd.read_csv('/Users/yichingchan/Documents/91APP/twice_above_sum.csv')
three_above_sum = pd.read_csv('/Users/yichingchan/Documents/91APP/three_above_sum.csv')
four_above_sum = pd.read_csv('/Users/yichingchan/Documents/91APP/four_above_sum.csv')
five_above_sum = pd.read_csv('/Users/yichingchan/Documents/91APP/five_above_sum.csv')

In [175]:
model = RandomForestClassifier()
d = dict()

accy_lst = []
f1_lst = []
recall_lst = []
precision_lst = []
ap_lst = []
#conf_m_lst = []
feature_lst = []

df_lst = [one_above_sum, twice_above_sum, three_above_sum, four_above_sum, five_above_sum]
for df in df_lst:
    pred_Y, probs_Y, accy, f1, recall, precision, conf_m, ap, feature_importances = binary_classifier(df, model)
    accy_lst.append(accy)
    f1_lst.append(f1)
    recall_lst.append(recall)
    precision_lst.append(precision)
    ap_lst.append(ap)
    #conf_m_lst.append(conf_m)
    feature_lst.append(feature_importances.to_dict())
    
d['RF_accy'] = accy_lst
d['RF_f1'] = f1_lst
d['RF_recall'] = recall_lst
d['RF_precision'] = precision_lst
d['RF_ap'] = ap_lst
#LR['conf_m'] = conf_m_lst
RF_matrix = pd.DataFrame(d)
RF_matrix.index = ['oneMore', 'twoMore', 'threeMore', 'fourMore', 'fiveMore']

RFfeature_rank = pd.DataFrame(feature_lst)
RFfeature_mean_score = RFfeature_rank.mean().sort_values(ascending=False)

False    184348
True      72142
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.726, 
 auc: 0.666, 
 f1: 0.381, 
 recall: 0.596, 
 precision: 0.526

confusion matrix :        False  True 
False  32922   3898
True   10160   4318

ap: 0.438
False    47381
True     37720
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.622, 
 auc: 0.644, 
 f1: 0.508, 
 recall: 0.603, 
 precision: 0.596

confusion matrix :        False  True 
False   7270   2249
True    4180   3322

ap: 0.574
True     21272
False    16860
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.596, 
 auc: 0.633, 
 f1: 0.622, 
 recall: 0.595, 
 precision: 0.643

confusion matrix :        False  True 
False   2007   1410
True    1675   2535

ap: 0.655
True     12650
False     7156
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.640, 
 auc: 0.644, 
 f1: 0.725, 
 recall: 0.600, 
 precision: 0.712

confusion matrix :        False  True 
False    651    761
True     667   18

In [176]:
model = LogisticRegression()
d = dict()

accy_lst = []
f1_lst = []
recall_lst = []
precision_lst = []
ap_lst = []
#conf_m_lst = []


df_lst = [one_above_sum, twice_above_sum, three_above_sum, four_above_sum, five_above_sum]
for df in df_lst:
    pred_Y, probs_Y, accy, f1, recall, precision, conf_m, ap, feature_importances = binary_classifier(df, model)
    accy_lst.append(accy)
    f1_lst.append(f1)
    recall_lst.append(recall)
    precision_lst.append(precision)
    ap_lst.append(ap)
    #conf_m_lst.append(conf_m)
    
    
d['LR_accy'] = accy_lst
d['LR_f1'] = f1_lst
d['LR_recall'] = recall_lst
d['LR_precision'] = precision_lst
d['LR_ap'] = ap_lst
#LR['conf_m'] = conf_m_lst
LR_matrix = pd.DataFrame(d)
LR_matrix.index = ['oneMore', 'twoMore', 'threeMore', 'fourMore', 'fiveMore']


False    184348
True      72142
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.755, 
 auc: 0.727, 
 f1: 0.380, 
 recall: 0.607, 
 precision: 0.688

confusion matrix :        False  True 
False  34902   1745
True   10804   3847

ap: 0.555
False    47381
True     37720
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.655, 
 auc: 0.700, 
 f1: 0.567, 
 recall: 0.640, 
 precision: 0.641

confusion matrix :        False  True 
False   7304   2149
True    3727   3841

ap: 0.658
True     21272
False    16860
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.650, 
 auc: 0.689, 
 f1: 0.710, 
 recall: 0.633, 
 precision: 0.666

confusion matrix :        False  True 
False   1683   1642
True    1030   3272

ap: 0.735
True     12650
False     7156
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.668, 
 auc: 0.669, 
 f1: 0.767, 
 recall: 0.598, 
 precision: 0.695

confusion matrix :        False  True 
False    491    945
True     369   21

In [264]:
model = XGBClassifier()
d = dict()

accy_lst = []
f1_lst = []
recall_lst = []
precision_lst = []
ap_lst = []
#conf_m_lst = []
feature_lst = []

df_lst = [one_above_sum, twice_above_sum, three_above_sum, four_above_sum, five_above_sum]
for df in df_lst:
    pred_Y, probs_Y, accy, f1, recall, precision, conf_m, ap, feature_importances = binary_classifier(df, model)
    accy_lst.append(accy)
    f1_lst.append(f1)
    recall_lst.append(recall)
    precision_lst.append(precision)
    ap_lst.append(ap)
    #conf_m_lst.append(conf_m)
    feature_lst.append(feature_importances.to_dict())
    
d['XGB_accy'] = accy_lst
d['XGB_f1'] = f1_lst
d['XGB_recall'] = recall_lst
d['XGB_precision'] = precision_lst
d['XGB_ap'] = ap_lst
#LR['conf_m'] = conf_m_lst
XGB_matrix = pd.DataFrame(d)
XGB_matrix.index = ['oneMore', 'twoMore', 'threeMore', 'fourMore', 'fiveMore']

XGBfeature_rank = pd.DataFrame(feature_lst)
XGBfeature_mean_score = XGBfeature_rank.mean().sort_values(ascending=False)

False    184348
True      72142
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.761, 
 auc: 0.731, 
 f1: 0.405, 
 recall: 0.617, 
 precision: 0.671

confusion matrix :        False  True 
False  34836   2051
True   10230   4181

ap: 0.559
False    47381
True     37720
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.663, 
 auc: 0.705, 
 f1: 0.571, 
 recall: 0.646, 
 precision: 0.649

confusion matrix :        False  True 
False   7451   2073
True    3671   3826

ap: 0.670
True     21272
False    16860
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.644, 
 auc: 0.697, 
 f1: 0.700, 
 recall: 0.631, 
 precision: 0.656

confusion matrix :        False  True 
False   1746   1657
True    1059   3165

ap: 0.741
True     12650
False     7156
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.680, 
 auc: 0.698, 
 f1: 0.776, 
 recall: 0.608, 
 precision: 0.702

confusion matrix :        False  True 
False    497    933
True     335   21

In [266]:
print('top 10 feature of XGB model:')
print()
print(pd.DataFrame(feature_lst).transpose().sum(axis=1).sort_values(ascending=False).head(10))

top 10 feature of XGB model:

season_spring                  1.537210
season_winter                  0.762382
get_LastBirthdayPresentYear    0.498456
fre_mean                       0.295182
SalesOrderSlaveTotalPayment    0.291555
get_OpenCardPresent            0.241042
season_summer                  0.152745
TrackDeviceTypeDef_Mobile      0.132805
TrackSourceTypeDef_Web         0.099542
ECouponDiscount                0.087226
dtype: float64


In [267]:
model = GradientBoostingClassifier()
d = dict()

accy_lst = []
f1_lst = []
recall_lst = []
precision_lst = []
ap_lst = []
#conf_m_lst = []
feature_lst = []

df_lst = [one_above_sum, twice_above_sum, three_above_sum, four_above_sum, five_above_sum]
for df in df_lst:
    pred_Y, probs_Y, accy, f1, recall, precision, conf_m, ap, feature_importances = binary_classifier(df, model)
    accy_lst.append(accy)
    f1_lst.append(f1)
    recall_lst.append(recall)
    precision_lst.append(precision)
    ap_lst.append(ap)
    #conf_m_lst.append(conf_m)
    feature_lst.append(feature_importances.to_dict())
    
d['GB_accy'] = accy_lst
d['GB_f1'] = f1_lst
d['GB_recall'] = recall_lst
d['GB_precision'] = precision_lst
d['GB_ap'] = ap_lst
#LR['conf_m'] = conf_m_lst
GB_matrix = pd.DataFrame(d)
GB_matrix.index = ['oneMore', 'twoMore', 'threeMore', 'fourMore', 'fiveMore']

GBfeature_rank = pd.DataFrame(feature_lst)
GBfeature_mean_score = GBfeature_rank.mean().sort_values(ascending=False)

False    184348
True      72142
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.760, 
 auc: 0.731, 
 f1: 0.413, 
 recall: 0.620, 
 precision: 0.664

confusion matrix :        False  True 
False  34650   2186
True   10133   4329

ap: 0.560
False    47381
True     37720
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.654, 
 auc: 0.705, 
 f1: 0.556, 
 recall: 0.637, 
 precision: 0.646

confusion matrix :        False  True 
False   7443   2020
True    3867   3691

ap: 0.669
True     21272
False    16860
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.651, 
 auc: 0.704, 
 f1: 0.705, 
 recall: 0.638, 
 precision: 0.668

confusion matrix :        False  True 
False   1775   1583
True    1080   3189

ap: 0.751
True     12650
False     7156
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.674, 
 auc: 0.690, 
 f1: 0.775, 
 recall: 0.598, 
 precision: 0.690

confusion matrix :        False  True 
False    453    995
True     295   22

In [268]:
print('top 10 feature of GB model:')
print()
print(pd.DataFrame(feature_lst).transpose().sum(axis=1).sort_values(ascending=False).head(10))

top 10 feature of GB model:

season_spring                  1.670847
fre_mean                       0.798535
SalesOrderSlaveTotalPayment    0.731351
season_winter                  0.377458
get_OpenCardPresent            0.365416
get_LastBirthdayPresentYear    0.348498
mean_payment                   0.148715
TrackSourceTypeDef_Web         0.073189
TrackDeviceTypeDef_Mobile      0.070905
season_summer                  0.053060
dtype: float64


In [269]:
one_above_sum.head()

,MemberId,Quantity,SalesOrderSlaveTotalPayment,"cut_UnitPrice_(399.0, 3500.0]","cut_UnitPrice_(311.0, 399.0]","cut_UnitPrice_(235.0, 311.0]","cut_UnitPrice_(199.0, 235.0]","cut_UnitPrice_(0.0, 199.0]",ECouponDiscount,PromotionDiscount,location_North,location_South,location_Mid,location_East,Weekend,time_Midnight,time_Morning,time_Afternoon,time_Night,season_spring,season_summer,season_fall,season_winter,TrackSourceTypeDef_AndroidApp,TrackSourceTypeDef_Web,TrackSourceTypeDef_iOSApp,TrackDeviceTypeDef_Mobile,TrackDeviceTypeDef_PC,TrackDeviceTypeDef_Pad,PayProfileTypeDef_ATM,PayProfileTypeDef_CreditCardOnce,PayProfileTypeDef_Family,PayProfileTypeDef_LinePay,PayProfileTypeDef_SevenEleven,DeliverTypeDef_Family,DeliverTypeDef_Home,DeliverTypeDef_SevenEleven,count,get_OpenCardPresent,get_LastBirthdayPresentYear,fre_mean,mean_payment,label,ctrbn,retain
0,1362,4,591.0,0,0,1,0,3,0.0,0.0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,True,True,365.0,147.750000,Loss,Loss,False
1,1384,1,590.0,1,0,0,0,0,0.0,0.0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,False,False,365.0,590.000000,C,tier 4,True
2,1400,2,798.0,0,2,0,0,0,0.0,0.0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,False,False,365.0,399.000000,Loss,Loss,False
3,1448,9,2112.0,0,2,2,2,3,0.0,0.0,0,0,2,0,1,0,0,1,1,0,2,0,0,0,2,0,1,1,0,0,0,0,0,2,0,0,2,2,False,False,66.0,240.166667,D,tier 2,True
4,1469,9,2768.0,2,3,2,0,2,0.0,2.0,2,2,0,0,1,3,1,0,0,0,2,1,1,4,0,0,4,0,0,0,0,2,0,2,2,0,2,4,True,True,56.0,331.175000,Loss,Loss,False


In [179]:
model = GaussianNB()
d = dict()

accy_lst = []
f1_lst = []
recall_lst = []
precision_lst = []
ap_lst = []
#conf_m_lst = []

df_lst = [one_above_sum, twice_above_sum, three_above_sum, four_above_sum, five_above_sum]
for df in df_lst:
    pred_Y, probs_Y, accy, f1, recall, precision, conf_m, ap, feature_importances = binary_classifier(df, model)
    accy_lst.append(accy)
    f1_lst.append(f1)
    recall_lst.append(recall)
    precision_lst.append(precision)
    ap_lst.append(ap)
    #conf_m_lst.append(conf_m)
    
d['NB_accy'] = accy_lst
d['NB_f1'] = f1_lst
d['NB_recall'] = recall_lst
d['NB_precision'] = precision_lst
d['NB_ap'] = ap_lst
#LR['conf_m'] = conf_m_lst
NB_matrix = pd.DataFrame(d)
NB_matrix.index = ['oneMore', 'twoMore', 'threeMore', 'fourMore', 'fiveMore']

False    184348
True      72142
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.729, 
 auc: 0.668, 
 f1: 0.392, 
 recall: 0.602, 
 precision: 0.534

confusion matrix :        False  True 
False  32938   3911
True    9970   4479

ap: 0.461
False    47381
True     37720
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.622, 
 auc: 0.671, 
 f1: 0.412, 
 recall: 0.587, 
 precision: 0.650

confusion matrix :        False  True 
False   8339   1215
True    5214   2253

ap: 0.609
True     21272
False    16860
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.573, 
 auc: 0.661, 
 f1: 0.505, 
 recall: 0.593, 
 precision: 0.699

confusion matrix :        False  True 
False   2709    717
True    2539   1662

ap: 0.684
True     12650
False     7156
Name: retain, dtype: int64

evaluation without RFECV: 
 accy: 0.613, 
 auc: 0.664, 
 f1: 0.665, 
 recall: 0.615, 
 precision: 0.736

confusion matrix :        False  True 
False    906    545
True     990   15

In [180]:
df_matrix_sum = pd.concat([LR_matrix, RF_matrix, XGB_matrix, GB_matrix, NB_matrix ], axis=1)
df_matrix_sum

,LR_accy,LR_f1,LR_recall,LR_precision,LR_ap,RF_accy,RF_f1,RF_recall,RF_precision,RF_ap,XGB_accy,XGB_f1,XGB_recall,XGB_precision,XGB_ap,GB_accy,GB_f1,GB_recall,GB_precision,GB_ap,NB_accy,NB_f1,NB_recall,NB_precision,NB_ap
oneMore,0.755371,0.380082,0.607480,0.687947,0.555020,0.725954,0.380541,0.596190,0.525560,0.438191,0.762856,0.411722,0.620266,0.671663,0.563912,0.760985,0.414050,0.620695,0.663705,0.560632,0.729405,0.392224,0.601925,0.533850,0.461356
twoMore,0.654779,0.566603,0.640098,0.641235,0.658409,0.622290,0.508223,0.603275,0.596302,0.573845,0.659891,0.565097,0.643084,0.647332,0.666134,0.656659,0.564076,0.641163,0.649768,0.668292,0.622290,0.412071,0.587278,0.649654,0.609106
threeMore,0.649666,0.710069,0.633371,0.665853,0.734708,0.595516,0.621704,0.594748,0.642586,0.654606,0.656615,0.709742,0.643894,0.674389,0.754820,0.659630,0.712195,0.647186,0.673658,0.750104,0.573096,0.505167,0.593169,0.698613,0.684371
fourMore,0.668349,0.766525,0.597921,0.695358,0.768568,0.639576,0.725067,0.599740,0.712179,0.742804,0.679707,0.775438,0.610537,0.692916,0.792965,0.691065,0.784507,0.616920,0.714790,0.810456,0.612569,0.664627,0.615066,0.736205,0.756260
fiveMore,0.729969,0.828459,0.595433,0.746579,0.819297,0.688358,0.786148,0.602451,0.765228,0.798137,0.728198,0.828874,0.582060,0.757900,0.837267,0.711819,0.816775,0.576691,0.729879,0.832980,0.676848,0.760656,0.639322,0.790729,0.809756


In [181]:
mod = ['LR_', 'RF_', 'XGB_', 'GB_', 'NB_']
eva = ['accy', 'f1', 'recall', 'precision', 'ap']
accy = pd.DataFrame()
f1 = pd.DataFrame()
recall = pd.DataFrame()
precision = pd.DataFrame()
ap = pd.DataFrame()

df = [accy, f1, recall, precision, ap]

for name in mod:
    for index in range(len(eva)):
        sr = name + eva[index]
        df[index][sr] = df_matrix_sum[sr]


In [182]:
accy.index = ['purchase>1','purchase>2','purchase>3','purchase>4','purchase>5']
accy


,LR_accy,RF_accy,XGB_accy,GB_accy,NB_accy
purchase>1,0.755371,0.725954,0.762856,0.760985,0.729405
purchase>2,0.654779,0.622290,0.659891,0.656659,0.622290
purchase>3,0.649666,0.595516,0.656615,0.659630,0.573096
purchase>4,0.668349,0.639576,0.679707,0.691065,0.612569
purchase>5,0.729969,0.688358,0.728198,0.711819,0.676848


In [183]:
f1

,LR_f1,RF_f1,XGB_f1,GB_f1,NB_f1
oneMore,0.380082,0.380541,0.411722,0.414050,0.392224
twoMore,0.566603,0.508223,0.565097,0.564076,0.412071
threeMore,0.710069,0.621704,0.709742,0.712195,0.505167
fourMore,0.766525,0.725067,0.775438,0.784507,0.664627
fiveMore,0.828459,0.786148,0.828874,0.816775,0.760656


In [184]:
recall

,LR_recall,RF_recall,XGB_recall,GB_recall,NB_recall
oneMore,0.607480,0.596190,0.620266,0.620695,0.601925
twoMore,0.640098,0.603275,0.643084,0.641163,0.587278
threeMore,0.633371,0.594748,0.643894,0.647186,0.593169
fourMore,0.597921,0.599740,0.610537,0.616920,0.615066
fiveMore,0.595433,0.602451,0.582060,0.576691,0.639322


In [185]:
precision

,LR_precision,RF_precision,XGB_precision,GB_precision,NB_precision
oneMore,0.687947,0.525560,0.671663,0.663705,0.533850
twoMore,0.641235,0.596302,0.647332,0.649768,0.649654
threeMore,0.665853,0.642586,0.674389,0.673658,0.698613
fourMore,0.695358,0.712179,0.692916,0.714790,0.736205
fiveMore,0.746579,0.765228,0.757900,0.729879,0.790729


In [186]:
ap.index = ['purchase>1','purchase>2','purchase>3','purchase>4','purchase>5']
ap

,LR_ap,RF_ap,XGB_ap,GB_ap,NB_ap
purchase>1,0.555020,0.438191,0.563912,0.560632,0.461356
purchase>2,0.658409,0.573845,0.666134,0.668292,0.609106
purchase>3,0.734708,0.654606,0.754820,0.750104,0.684371
purchase>4,0.768568,0.742804,0.792965,0.810456,0.756260
purchase>5,0.819297,0.798137,0.837267,0.832980,0.809756


In [187]:
feaRank_sum = []
feaRank_sum.append(RFfeature_mean_score.to_dict())
feaRank_sum.append(XGBfeature_mean_score.to_dict())
feaRank_sum.append(GBfeature_mean_score.to_dict())
featureRanking_sum = pd.DataFrame(feaRank_sum)
featureRanking_sum.index = ['RF', 'XGB', 'GB']
featureRanking_sum.transpose()

,RF,XGB,GB
DeliverTypeDef_Family,0.008858,0.000000,0.000278
DeliverTypeDef_Home,0.004826,0.000000,0.000339
DeliverTypeDef_SevenEleven,0.009682,0.000000,0.000223
ECouponDiscount,0.013798,0.016820,0.008753
PayProfileTypeDef_ATM,0.000014,0.000000,0.000010
PayProfileTypeDef_CreditCardOnce,0.005228,0.006631,0.000669
PayProfileTypeDef_Family,0.008831,0.002765,0.000435
PayProfileTypeDef_LinePay,0.000000,0.000000,0.000000
PayProfileTypeDef_SevenEleven,0.008827,0.003846,0.000315
PromotionDiscount,0.018704,0.016448,0.004384


In [188]:
featureRanking_sum.transpose()['GB'].sort_values(ascending=False).head(10)

season_spring                  0.341232
fre_mean                       0.152224
SalesOrderSlaveTotalPayment    0.142749
season_winter                  0.075052
get_OpenCardPresent            0.073170
get_LastBirthdayPresentYear    0.072197
mean_payment                   0.032849
TrackSourceTypeDef_Web         0.015087
TrackDeviceTypeDef_Mobile      0.014889
season_summer                  0.010815
Name: GB, dtype: float64

In [189]:
featureRanking_sum.transpose()['RF'].sort_values(ascending=False).head(10)

SalesOrderSlaveTotalPayment      0.119152
mean_payment                     0.111062
fre_mean                         0.102697
Quantity                         0.070508
cut_UnitPrice_(0.0, 199.0]       0.067461
cut_UnitPrice_(235.0, 311.0]     0.059168
cut_UnitPrice_(311.0, 399.0]     0.051379
cut_UnitPrice_(399.0, 3500.0]    0.036871
season_spring                    0.032405
cut_UnitPrice_(199.0, 235.0]     0.028527
Name: RF, dtype: float64

In [190]:
featureRanking_sum.transpose()['XGB'].sort_values(ascending=False).head(10)

season_spring                  0.319783
season_winter                  0.145660
get_LastBirthdayPresentYear    0.096631
SalesOrderSlaveTotalPayment    0.061298
fre_mean                       0.055194
get_OpenCardPresent            0.047013
season_summer                  0.031539
TrackDeviceTypeDef_Mobile      0.026944
TrackSourceTypeDef_Web         0.020863
ECouponDiscount                0.016820
Name: XGB, dtype: float64

###  預期貢獻

In [191]:
expected_value = pd.read_csv('/Users/yichingchan/Documents/91APP/expected_value.csv')

In [207]:
expected_value = expected_value.fillna(0)

In [225]:
train_Y = expected_value[['retain','monetary ctrbn','MemberId']]
drop_list =['MemberId', 'count', 'label', 'ctrbn', 'retain','monetary ctrbn']
train_X = expected_value.drop(drop_list ,axis=1)
column_name = train_X.columns.tolist()

train_X = train_X.values
train_X, test_X, train_Y, test_Y = train_test_split(train_X, train_Y, test_size=0.2)

In [226]:
#預測留存率

#model fit
model = XGBClassifier()
model.fit(train_X, train_Y['retain'])
 

    ##predict
pred_Y = model.predict(test_X)
probs_Y = model.predict_proba(test_X)
probs_Y = probs_Y[:, 1] ## keep probabilities for the positive outcome only      

In [243]:
expect = test_Y
expect['prob'] = probs_Y.round(2)
#expect['pred'] = pred_Y
expect.head()

,retain,monetary ctrbn,MemberId,prob,pred,pred value
30691,True,4385.0,3268332,0.66,True,2108.420898
19854,True,9588.0,1954408,0.87,True,5316.116699
22657,False,0.0,2400795,0.68,True,1916.297119
13397,True,2445.0,1373770,0.59,True,1854.556641
34705,True,8216.0,3788818,0.91,True,4043.693359


In [228]:
#預測貢獻值
import xgboost
from xgboost import XGBRegressor

#model fit
model = XGBRegressor()
model.fit(train_X, train_Y['monetary ctrbn'])
 

##predict
pred_Y = model.predict(test_X)
#probs_Y = model.predict_proba(test_X)
#probs_Y = probs_Y[:, 1] ## keep probabilities for the positive outcome only      

[02:17:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [246]:
temp = test_Y
temp['pred value'] = pred_Y
temp.head()

,retain,monetary ctrbn,MemberId,prob,pred,pred value
30691,True,4385.0,3268332,0.66,True,2108.420898
19854,True,9588.0,1954408,0.87,True,5316.116699
22657,False,0.0,2400795,0.68,True,1916.297119
13397,True,2445.0,1373770,0.59,True,1854.556641
34705,True,8216.0,3788818,0.91,True,4043.693359


In [255]:
result = pd.merge(expect[['MemberId', 'prob']], temp[['MemberId', 'pred value']], on='MemberId', how='left')
result['expected value'] = result['prob'] * result['pred value']
result.sort_values(by='expected value', ascending=False).head(10).round(2)

,MemberId,prob,pred value,expected value
4968,3385867,0.88,31013.529297,27291.910156
282,3287557,0.95,24677.500000,23443.630859
2784,2841266,0.95,21618.359375,20537.449219
1600,620298,0.94,21767.099609,20461.070312
6784,595264,0.96,21226.480469,20377.419922
4880,3091561,0.94,17713.470703,16650.660156
5061,1697765,0.96,17300.480469,16608.460938
3224,1749824,0.93,16244.759766,15107.620117
3847,2988895,0.95,15592.570312,14812.940430
1363,2932837,0.92,15997.570312,14717.759766


In [256]:
a = []
for i in result['pred value'].tolist():
    k = round(i, 2)
    a.append(k)
b = []
for i in result['expected value'].tolist():
    k = round(i, 2)
    b.append(k)
result['pred value'] = a
result['expected value'] = b

In [257]:
result.sort_values(by='expected value', ascending=False).head(10).round(2)

,MemberId,prob,pred value,expected value
4968,3385867,0.88,31013.53,27291.91
282,3287557,0.95,24677.51,23443.63
2784,2841266,0.95,21618.37,20537.45
1600,620298,0.94,21767.10,20461.07
6784,595264,0.96,21226.48,20377.42
4880,3091561,0.94,17713.47,16650.66
5061,1697765,0.96,17300.48,16608.46
3224,1749824,0.93,16244.76,15107.62
3847,2988895,0.95,15592.57,14812.94
1363,2932837,0.92,15997.57,14717.76
